# 经典案例1：最少硬币个数的问题

给定一定的硬币组合，如何在找零时找给客户最少数量硬币

方案1：贪心策略
- 从最大面值的硬币开始，用尽量多数量的硬币找零
- 有余额时，再使用下一个最大面值的硬币，重复上一步
- 直到余额为零

这个策略在常规的货币体系下表现较好，但是假设提供的硬币组合是{1,5,10,21,25}，在面对找零为63的时候，贪心策略会失效

### 解决该问题的递归解法

对于63个硬币，减小问题规模，相当于在以下5个方案中寻找最小硬币数解：

1. 1枚1元硬币+62个硬币的最小硬币数（62个硬币最优方案为递归展开）
2. 1枚5元硬币+58个硬币的最小硬币数（58个硬币最优方案为递归展开）
3. 1枚10元硬币+53个硬币的最小硬币数（53个硬币最优方案为递归展开）
4. 1枚21元硬币+42个硬币的最小硬币数（42个硬币最优方案为递归展开）
5. +1枚25元硬币+38个硬币的最小硬币数（38个硬币最优方案为递归展开）

In [2]:
def recMC(coinValueList, change):
    minCoins = change #先最大化找零硬币数
    if change in coinValueList:
        return 1
    else:
        for i in [c for c in coinValueList if c <= change]: #单枚硬币价值大于剩余找零的就不进入迭代
            numCoins = 1 + recMC(coinValueList, change - i) #递归得到最小硬币数
            if numCoins < minCoins:
                minCoins = numCoins #更新最小硬币数
    return minCoins

In [4]:
#test
import time
print(time.clock()) #开始时间
print(recMC([1,5,10,21,25], 63))
print(time.clock()) #结束时间

5.476737
3
23.505405


由上面的测试就可以发现该递归算法非常低效，因为其递归存在大量重复（5条路径下的递归存在重复的最优解计算）

优化该算法的办法是在过程中，记录最优解，避免重复计算

In [10]:
def recDC(coinValueList, change, knownResults): #增加knownResults记录已经计算过的最优值
    minCoins = change #先最大化找零硬币数
    if change in coinValueList:
        knownResults[change] = 1 #记录硬币种类对应零钱数的最优解
        return 1
    elif knownResults[change] > 0:
        return knownResults[change] #返回已经记录的最优解硬币个数
    else:
        for i in [c for c in coinValueList if c <= change]: 
            numCoins = 1 + recDC(coinValueList, change - i, knownResults) 
            if numCoins < minCoins:
                minCoins = numCoins 
                knownResults[change] = minCoins #更新找零对应的最小硬币个数
    return minCoins

In [11]:
#test
print(time.clock())
print(recDC([1,5,10,21,25], 63, [0]*64))
print(time.clock())

24.993112
3
24.995501


通过记录过程，可见整个运算时间变化非常明显
这种技术被称为**记忆化/函数值缓存**(memoization)

### 方案二：动态规划

动态规划使用的必要条件：**问题的最优解包含了更小规模子问题的最优解**

- 动态规划算法采用了**更有条理**的方式来求解
- 在找零兑换问题上，动态规划会从**最简单**的1分钱找零开始，逐步增加找零数，直至我们需要求解的找零数
- 在增加找零的过程中，设法保证每一分钱的增加都是最优解，这样在达到所求零钱数时，自然得到最优解



In [14]:
def dpMakeChange(coinValueList, change, minCoins): #minCoins为记录从1到零钱数所有情况的最少硬币数
    for cents in range(1, change+1): #从1分钱计算到找零需要的钱数
        coinCount = cents
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents - j] + 1 < coinCount: #如果更小规模的最优解加上一枚硬币可以得到该找零额度的最优解
                coinCount = minCoins[cents - j] + 1 #更新最优解
        minCoins[cents] = coinCount #最优解记录到表格中
    return minCoins[change] #返回要求找零数的结果

In [16]:
#test
print(time.clock())
print(dpMakeChange([1,5,10,21,25], 63, [0]*64))
print(time.clock())

28.038844
3
28.040853


如果需要获得最优解的硬币组合，那么需要在最优解的表格中，记录每个零钱数相较于上一步增加的硬币类型

In [21]:
def dpMakeChange2(coinValueList, change, minCoins, coinUsed): #增加表格记录每一步对应的硬币
    for cents in range(1, change+1): 
        coinCount = cents
        newCoin = 1 #初始化硬币面额
        for j in [c for c in coinValueList if c <= cents]:
            if minCoins[cents - j] + 1 < coinCount: 
                coinCount = minCoins[cents - j] + 1
                newCoin = j #更新硬币面额
        minCoins[cents] = coinCount
        coinUsed[cents] = newCoin
    return minCoins[change]

def printCoins(coinsUsed, change): #建立函数打印出使用的硬币面额
    coin = change
    l = []
    while coin > 0:
        thisCoin = coinsUsed[coin]
        l.append(thisCoin)
        coin = coin - thisCoin
    return l

In [22]:
#test
amnt = 63
clist = [1, 5, 10, 21, 25]
coinUsed = [0] * (amnt + 1)
coinCount = [0] * (amnt + 1)

print(dpMakeChange2(clist, amnt, coinCount, coinUsed))
print(f"They are {printCoins(coinUsed, amnt)}")

3
They are [21, 21, 21]


# 经典案例2：博物馆大盗
假设大盗负重20公斤，博物馆内宝物价值和重量如下表

| 编号 | 重量 | 价值 |
|:--:|:--:|:--:|
|1|2|3|
|2|3|4|
|3|4|8|
|4|5|8|
|5|9|10|

可以将动态规划的解法描述如下：

有函数m(i, W)，其表示为 前i(1<=i<=5)个宝物中，组合不超过W(1<=W<=20)重量，可以得到的最大价值
- m(i, W)是m(i-1, W)和m(i-1, W-Wi)+vi两者的最大值
- 从m(1,1)计算到m(5,20)
- 当i=0或W=0时，m(i, W)=0
- 当Wi > W时，m(i, W)=m(i-1, W)



In [1]:
#动态规划解法
tr = [
    None, {'w':2, 'v':3}, {'w':3, 'v':4},
    {'w':4, 'v':8}, {'w':5, 'v':8}, {'w':9, 'v':10}
] #构建宝物清单

max_w = 20

#初始化二维价值表格m(i, W)
m = {(i, w):0 for i in range(len(tr)) 
    for w in range(max_w + 1)}

for i in range(1, len(tr)):
    for w in range(1, max_w + 1):
        if tr[i]['w'] > w: #装不下该宝物
            m[(i, w)] = m[(i-1, w)] #价值表格该格以少一件的价值来填写
        else:
            m[(i, w)] = max(m[(i-1, w)], m[i-1, w-tr[i]['w']] + tr[i]['v'])

print(m[(len(tr)-1, max_w)])

29


In [3]:
#递归解法
tr = {(2, 3), (3, 4), (4, 8), (5, 8), (9, 10)}
max_w = 20

m = {} #初始化记录表格，key是（宝物组合，最大重量），value是最大价值

def thief(tr, w):
    if tr == set() or w == 0: #结束条件
        m[(tuple(tr), w)] = 0 #key必须为tuple
        return 0
    elif (tuple(tr), w) in m:
        return m[(tuple(tr), w)]
    else:
        vmax = 0
        for t in tr:
            if t[0] <= w:
                #逐个从集合中去掉某个宝物，递归调用
                #选出所有价值中的最大值
                v = thief(tr-{t}, w-t[0]) + t[1]
                vmax = max(vmax, v)
            m[(tuple(tr), w)] = vmax
        return vmax

print(thief(tr, max_w))

29
